In [16]:
from sqlalchemy import (
    Column, Integer, String, Text, ForeignKey, DateTime, Boolean, Enum, Index, Table
)
from sqlalchemy.orm import relationship, declarative_base, Session
from sqlalchemy import create_engine

from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import func
from datetime import datetime
from sqlalchemy.dialects.postgresql import JSONB
from dotenv import load_dotenv
import os
import boto3
import enum

# Step 0. Imports & Base

In [ ]:
# from sqlalchemy import create_engine, text

# # load the .env file
# load_dotenv()

# # get database url
# DATABASE_URL = os.getenv("DATABASE_URL")

# engine = create_engine(DATABASE_URL, echo=True)

# with engine.connect() as conn:
#     result = conn.execute(text("SELECT version();"))
#     print("✅ Connected to:", result.scalar())

2025-09-21 14:00:55,055 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-09-21 14:00:55,055 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-21 14:00:55,056 INFO sqlalchemy.engine.Engine select current_schema()
2025-09-21 14:00:55,056 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-21 14:00:55,057 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-09-21 14:00:55,057 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-21 14:00:55,058 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 14:00:55,058 INFO sqlalchemy.engine.Engine SELECT version();
2025-09-21 14:00:55,058 INFO sqlalchemy.engine.Engine [generated in 0.00052s] {}
✅ Connected to: PostgreSQL 14.18 (Homebrew) on aarch64-apple-darwin23.6.0, compiled by Apple clang version 16.0.0 (clang-1600.0.26.6), 64-bit
2025-09-21 14:00:55,059 INFO sqlalchemy.engine.Engine ROLLBACK


### CAUTION: SQLAlchemy’s in-memory registry.
When you re-run your model definitions (class Project(Base) etc.), SQLAlchemy thinks you’re trying to define the same table again in the same Python session.

Fixes:

* Restart the kernel (clean slate).

* Or run Base.metadata.clear() before re-defining models.

* Or (not recommended for production, but useful in notebooks) add:    

`__table_args__ = {"extend_existing": True}`    

inside each model.    

**Potential problems of 3rd option-- Silent overwrites**    

If you redefine a model with different column definitions, SQLAlchemy will happily overwrite the in-memory Python mapping.    

But the database table itself is unchanged — unless you drop/recreate or run a migration.    

This can cause a mismatch: your Python code thinks a column exists (or has a new type), but the real Postgres table does not.    

### My own suggestion:    
#### During prototyping stage:         
* use `__table_args__ = {"extend_existing": True}` for each model;
    
#### When your schema stabilizes and you’re preparing for AWS deployment:    
1. Move your models into models.py (or a models/ package).

        Delete __table_args__ = {"extend_existing": True} from each model.
        Define Base = declarative_base() once at the top.

2. Add Alembic to manage schema evolution:

    `pip install alembic`    

    `alembic init migrations`    

* Configure alembic.ini with your DATABASE_URL.   

* In env.py, set target_metadata = Base.metadata.

3. Whenever you change a model:

    `alembic revision --autogenerate -m "describe change"`
   
    `alembic upgrade head`

This will safely apply only the changes needed, without dropping your tables.

In [3]:
# create base class
Base = declarative_base() 

#---
# It creates a registry (Base.metadata) that will hold all the tables you define.
# Every time you define a model (class Project(Base): ...), that model’s table gets registered into Base.metadata.tables.
#---

In [ ]:
# Base.metadata.clear()

# Step 1. Enums

In [4]:
# --------------------------
# Enums
# --------------------------

## 1.1 ProjectStatus
class ProjectStatus(enum.Enum):
    draft = "draft"                  # project created, requirements being defined
    ready_for_annotation = "ready_for_annotation"  # files uploaded, jobs not started
    in_progress = "in_progress"      # annotation jobs are running
    completed = "completed"          # all jobs done
    archived = "archived"            # project closed, read-only

## 1.2 FileStatus (file lifecycle)
class FileStatus(enum.Enum):
    pending = "pending"
    ready_for_annotation = "ready_for_annotation"
    in_progress = "in_progress"
    completed = "completed"
    archived = "archived"

## 1.3 FileType
class FileType(enum.Enum):
    dataset = "dataset"
    requirement = "requirement"
    report = "annotation_results"
# Does our PM also needs to upload sliced file results? (NO currently)

## 1.4 UserRole
class UserRole(enum.Enum):
    org_admin = "org_admin"       # customer admin
    org_pm = "org_pm"             # customer project manager
    our_pm = "our_pm"             # our company PM that manages annotation jobs & assigns annotators
    annotator = "annotator"       # our company annotator
    qc = "qc"                     # our company QC for annotation results review

## 1.5 AnnotationJobStatus (job lifecycle)
class AnnotationJobStatus(enum.Enum):
    not_started = "not_started"
    in_progress = "in_progress"
    submitted = "submitted"
    reviewed = "reviewed"

## 1.6 ReviewStatus
class ReviewStatus(enum.Enum):
    pending = "pending"
    approved = "approved"
    rejected = "rejected"

## 1.7 EntityType
class EntityType(enum.Enum):
    project = "project"
    file = "file"
    file_version = "file_version"
    annotation_job = "annotation_job"

## 1.8 EventType
class EventType(enum.Enum):
    uploaded = "uploaded"
    reuploaded = "reuploaded"
    annotation_started = "annotation_started"
    annotation_completed = "annotation_completed"
    reviewed = "reviewed"
    deleted = "deleted"
    status_changed = "status_changed"

## 1.9 AssignmentRole
class AssignmentRole(enum.Enum):
    annotator = "annotator"
    reviewer = "reviewer"
    qc = "qc"   # quality control / audit

# Step 2. Association Tables 

In [5]:
# --------------------------
# Association Tables
# --------------------------

# 2.1 User <-> Role
user_roles = Table(
    "user_roles",
    Base.metadata,
    Column("user_id", Integer, ForeignKey("user.user_id", ondelete="CASCADE"), primary_key=True),
    Column("role_id", Integer, ForeignKey("role.role_id", ondelete="CASCADE"), primary_key=True)
)

# 2.2 Role <-> Permission
role_permissions = Table(
    "role_permissions",
    Base.metadata,
    Column("role_id", Integer, ForeignKey("role.role_id", ondelete="CASCADE"), primary_key=True),
    Column("permission_id", Integer, ForeignKey("permission.permission_id", ondelete="CASCADE"), primary_key=True)
)

# Step 3: CORE TABLES

## 3.1 Project Table

In [ ]:
# -----------------------------
# Core Tables
# -----------------------------
    
# Project Table
class Project(Base):
    __tablename__ = "project"
    __table_args__ = {"extend_existing": True}

    project_id = Column(Integer, primary_key=True, autoincrement=True)
    org_id = Column(Integer, ForeignKey("organization.org_id"), nullable=False)
    name = Column(String, nullable=False)


    # plain text instructions
    requirements_text = Column(Text, nullable=True)
    # optional uploaded doc (PDF, Word, PPT, etc.)
    # requirements_file_id = Column(Integer, ForeignKey("file.file_id"), nullable=True)

    # project status enum
    status = Column(Enum(ProjectStatus, name="project_status_enum"), default=ProjectStatus.draft)

    date_created = Column(DateTime, default=func.now(), nullable=False)
    date_updated = Column(DateTime, default=func.now(), onupdate=func.now(), nullable=False)

    files = relationship("File", back_populates="project") # all files under the project
    organization = relationship("Organization", back_populates="projects")
    client_pm_id = Column(Integer, ForeignKey("user.user_id"), nullable=False) # Associate with client PM
    requirement_files = relationship(
    "File",
    primaryjoin="and_(Project.project_id==File.project_id, File.file_type=='requirement')",
    viewonly=True) # only get files that are requirements


## 3.2 File Table 
(currently only create one for all kinds of files' storage)

In [ ]:
# --------------------------
# File Table
# --------------------------
class File(Base):
    __tablename__ = "file"
    __table_args__ = {"extend_existing": True}

    file_id = Column(Integer, primary_key=True, autoincrement=True)

    # belongs to a project
    project_id = Column(Integer, ForeignKey("project.project_id"), nullable=False)

    # descriptive file name (user-facing)
    name = Column(String, nullable=False)

    # what kind of file this is (dataset, requirement, annotation_results)
    file_type = Column(Enum(FileType, name="file_type_enum"), nullable=False, default=FileType.dataset)

    uploaded_by = Column(Integer, ForeignKey("user.user_id"), nullable=False)
    uploader = relationship("User", back_populates="uploaded_files")


    # workflow state
    status = Column(
        Enum(FileStatus, name="file_status_enum"),
        default=FileStatus.pending,
        nullable=False
    )

    # active version pointer
    active_version_id = Column(Integer, ForeignKey("file_version.version_id"), nullable=True)

    # audit timestamps
    date_created = Column(DateTime, default=func.now(), nullable=False)
    date_updated = Column(DateTime, default=func.now(), onupdate=func.now(), nullable=False)

    # --------------------------
    # Relationships
    # --------------------------
    project = relationship("Project", back_populates="files")
    versions = relationship("FileVersion", back_populates="file", cascade="all, delete-orphan")
    annotation_jobs = relationship("AnnotationJob", back_populates="file")


## 3.3 File Version Table

In [8]:
# File Version Table
class FileVersion(Base):
    __tablename__ = "file_version"
    __table_args__ = (
        Index("idx_fileversion_file_id", "file_id"),
        {"extend_existing": True},
    )

    version_id = Column(Integer, primary_key=True, autoincrement=True)
    file_id = Column(Integer, ForeignKey("file.file_id", ondelete="CASCADE"), nullable=False)
    s3_key = Column(String, nullable=False)  # MinIO/S3 object key
    uploaded_by = Column(Integer, ForeignKey("user.user_id"), nullable=True)
    uploaded_at = Column(DateTime(timezone=True), server_default=func.now(), nullable=False)
    is_active = Column(Boolean, default=True)

    # Relationships
    file = relationship("File", back_populates="versions")

## 3.4 User Table

"user" is a generic account table that represents any actor in the system:

* Organization Admin (client company, oversees all projects).

* Organization PM (client company, uploads datasets + requirements).

* Our PM (your company, manages annotation jobs & assigns annotators).

* Annotators (our company, upload results).

* QC / Reviewers (our company, upload corrections).

therefore, "user" here basically means everyone who logs in and interacts with the system.

In [ ]:
class User(Base):
    __tablename__ = "user"
    __table_args__ = {"extend_existing": True}


    user_id = Column(Integer, primary_key=True, autoincrement=True)
    email = Column(String, unique=True, nullable=False)
    role = Column(Enum(UserRole, name="user_role_enum"), nullable=False)
    org_id = Column(Integer, ForeignKey("organization.org_id"), nullable=True)  
    # org_id is only relevant for client users (admins, PMs)

    uploaded_files = relationship("File", back_populates="uploader")

    # Keep only one real relationship: User & EventLog
    events = relationship("EventLog", back_populates="user")
    assignments = relationship("Assignment", back_populates="user", cascade="all, delete-orphan")
    roles = relationship("Role", secondary=user_roles, back_populates="users")
    projects = relationship("Project", back_populates="client_pm") # associate with project


## 3.5 Annotation Job Table

In [10]:
# Annotation Job Table
class AnnotationJob(Base):
    __tablename__ = "annotation_job"
    __table_args__ = {"extend_existing": True}

    job_id = Column(Integer, primary_key=True, autoincrement=True)

    # Relationships
    file_id = Column(Integer, ForeignKey("file.file_id", ondelete="CASCADE"), nullable=False)
    project_id = Column(Integer, ForeignKey("project.project_id", ondelete="CASCADE"), nullable=False)
    assigned_to = Column(Integer, ForeignKey("user.user_id"), nullable=True)  # annotator
    reviewed_by = Column(Integer, ForeignKey("user.user_id"), nullable=True)  # reviewer

    # Workflow
    status = Column(
    Enum(AnnotationJobStatus, name="annotation_job_status_enum"),
    default=AnnotationJobStatus.not_started,
    nullable=False
    )
    
    # Review
    review_status = Column(
    Enum(ReviewStatus, name="review_status_enum"),
    default=ReviewStatus.pending,
    nullable=False
    )

    # Audit
    created_at = Column(DateTime, default=func.now(), nullable=False)
    updated_at = Column(DateTime, default=func.now(), onupdate=func.now(), nullable=False)

    # ORM relationships
    file = relationship("File", back_populates="annotation_jobs")
    project = relationship("Project", back_populates="annotation_jobs")
    annotator = relationship("User", foreign_keys=[assigned_to])
    reviewer = relationship("User", foreign_keys=[reviewed_by])
    reviews = relationship("Review", back_populates="job", cascade="all, delete-orphan")
    assignments = relationship("Assignment", back_populates="job", cascade="all, delete-orphan")

## 3.6 Event Log Table

In [11]:
# Event Log Table
class EventLog(Base):
    __tablename__ = "event_log"
    __table_args__ = {"extend_existing": True}

    event_id = Column(Integer, primary_key=True, autoincrement=True)

    entity_type = Column(Enum(EntityType, name="entity_type_enum"), nullable=False)
    entity_id = Column(Integer, nullable=False)      # e.g. file_id

    event_type = Column(Enum(EventType, name="event_type_enum"), nullable=False)

    user_id = Column(Integer, ForeignKey("user.user_id"), nullable=True)
    event_time = Column(DateTime, default=func.now())

    user = relationship("User", back_populates="events")

    event_metadata = Column(JSONB, nullable=True)  # use JSONB for flexible key/value storage


## 3.7 Review Table

In [12]:
# --------------------------
# Review Table
# --------------------------
class Review(Base):
    __tablename__ = "review"
    __table_args__ = {"extend_existing": True}

    review_id = Column(Integer, primary_key=True, autoincrement=True)

    # Link to the job being reviewed
    job_id = Column(Integer, ForeignKey("annotation_job.job_id"), nullable=False)

    # Reviewer (user with reviewer role)
    reviewer_id = Column(Integer, ForeignKey("user.user_id"), nullable=False)

    # Review decision
    status = Column(
        Enum(ReviewStatus, name="review_status_enum"),
        default=ReviewStatus.pending,
        nullable=False
    )

    # Optional comments from reviewer
    feedback = Column(Text, nullable=True)

    # Audit timestamps
    created_at = Column(DateTime, default=func.now(), nullable=False)
    updated_at = Column(DateTime, default=func.now(), onupdate=func.now(), nullable=False)

    # --------------------------
    # Relationships
    # --------------------------
    job = relationship("AnnotationJob", back_populates="reviews")
    reviewer = relationship("User")

## 3.8 Assignment Table

In [13]:
# --------------------------
# Assignment Table
# --------------------------
class Assignment(Base):
    __tablename__ = "assignment"
    __table_args__ = {"extend_existing": True}

    assignment_id = Column(Integer, primary_key=True, autoincrement=True)

    # Link to the annotation job
    job_id = Column(Integer, ForeignKey("annotation_job.job_id", ondelete="CASCADE"), nullable=False)

    # Who is assigned
    user_id = Column(Integer, ForeignKey("user.user_id", ondelete="CASCADE"), nullable=False)

    # Role in this job (annotator, reviewer, qc)
    role = Column(Enum(AssignmentRole, name="assignment_role_enum"), nullable=False)

    # Status of this assignment (separate from job status)
    status = Column(String, default="assigned")  
    # e.g. assigned, accepted, in_progress, completed

    # Audit fields
    assigned_at = Column(DateTime, default=func.now(), nullable=False)
    updated_at = Column(DateTime, default=func.now(), onupdate=func.now(), nullable=False)

    # Relationships
    job = relationship("AnnotationJob", back_populates="assignments")
    user = relationship("User", back_populates="assignments")
    

## 3.9 Role&Permission Table

In [14]:
# --------------------------
# Role Table
# --------------------------
class Role(Base):
    __tablename__ = "role"
    __table_args__ = {"extend_existing": True}

    role_id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, unique=True, nullable=False)   # e.g. "organization_admin", "pm", "annotator", "reviewer"

    # Relationships
    users = relationship("User", secondary=user_roles, back_populates="roles")
    permissions = relationship("Permission", secondary=role_permissions, back_populates="roles")

# --------------------------
# Permission Table
# --------------------------
class Permission(Base):
    __tablename__ = "permission"
    __table_args__ = {"extend_existing": True}

    permission_id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, unique=True, nullable=False)   # e.g. "upload_file", "assign_job", "review_annotation"

    # Relationships
    roles = relationship("Role", secondary=role_permissions, back_populates="permissions")